In [1]:
models_path = '/content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Modelling/Hasil Model'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import joblib

## Importing all models into notebook

### Weather Forecast

- 4 models in models_forecast
- forecast_future to forecast: def forecast_future(df, models, feature_sets, days_ahead=7)
- json feature: feature_forecast

In [70]:
import pickle
import os

# Correct directory to where .pkl models were saved
weather_models_dir = f"{models_path}/"
models_pkl = {}

# Ensure the directory exists
if not os.path.exists(weather_models_dir):
    print(f"Directory not found: {weather_models_dir}")
else:
    # Iterate through all files in the directory
    for filename in os.listdir(weather_models_dir):
        # Check for .pkl extension and specific filename prefix to load only our models
        if filename.endswith('.pkl') and filename.startswith('forecast_weather_xgboost_'):
            model_path = os.path.join(weather_models_dir, filename)
            model_name_without_ext = os.path.splitext(filename)[0] # e.g., 'forecast_weather_xgboost_rainfall_mm'

            # Load the model using pickle
            with open(model_path, 'rb') as f:
                loaded_model = pickle.load(f)
            models_pkl[filename] = loaded_model

print(f"Loaded {len(models_pkl)} models from {weather_models_dir}")

Loaded 4 models from /content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Modelling/Hasil Model/


In [71]:
# Load models used to forecast weather features

models_forecast = {}
for full_name, model_obj in models_pkl.items():
    # First, remove the .pkl extension
    name_without_ext = full_name.split('.')[0]
    # Then, remove the 'forecast_weather_xgboost_' prefix
    new_name = name_without_ext.replace('forecast_weather_xgboost_', '')
    models_forecast[new_name] = model_obj

print("Keys in models_pkl before renaming:", list(models_pkl.keys()))
print("Keys in models_forecast after renaming:", list(models_forecast.keys()))

Keys in models_pkl before renaming: ['forecast_weather_xgboost_rainfall_mm.pkl', 'forecast_weather_xgboost_humidity_pct.pkl', 'forecast_weather_xgboost_wind_speed_kmh.pkl', 'forecast_weather_xgboost_temperature_c.pkl']
Keys in models_forecast after renaming: ['rainfall_mm', 'humidity_pct', 'wind_speed_kmh', 'temperature_c']


In [72]:
# funstion to create features
def create_features(df, target, lags=[1,3,7], rolls=[3,7]):
    df = df.copy()

    # time-based
    df["month"] = df["date"].dt.month
    df["week"] = df["date"].dt.isocalendar().week.astype(int)
    df["dayofyear"] = df["date"].dt.dayofyear

    # lags per mine
    for lag in lags:
        df[f"{target}_lag{lag}"] = df.groupby("mine_id")[target].shift(lag)

    # rolling averages per mine
    for r in rolls:
        df[f"{target}_roll{r}"] = df.groupby("mine_id")[target].shift(1).rolling(r).mean()

    return df

In [73]:
# targets to forecast
targets = ["rainfall_mm", "temperature_c", "humidity_pct", "wind_speed_kmh"]

In [74]:
# Function to forecast each target
def forecast_future(df, models, feature_sets, days_ahead=7):
    df_future = df.copy()
    last_date = df["date"].max()

    future_rows = []

    for i in range(days_ahead):
        next_date = last_date + pd.Timedelta(days=i+1)

        for mine in df["mine_id"].unique():
            row = {
                "date": next_date,
                "mine_id": mine,
                "mine_id_enc": le.transform([mine])[0],
                "month": next_date.month,
                "week": next_date.isocalendar().week,
                "dayofyear": next_date.timetuple().tm_yday
            }

            temp_df = pd.concat([df_future, pd.DataFrame([row])], ignore_index=True)

            for target in targets:
                temp_df = create_features(temp_df, target)

                # ambil baris terakhir untuk predict
                pred_input = temp_df.iloc[-1:][feature_sets[target]]

                # predict
                row[target] = models[target].predict(pred_input)[0]

            #df_future = pd.concat([df_future, pd.DataFrame(row)], ignore_index=True, axis = 1)
            future_rows.append(row)

    return pd.DataFrame(future_rows)

In [75]:
# Import features for modelling
import json

with open(f"{models_path}/feature_forecast.json") as f:
    feature_forecast = json.load(f)

### Weather Classification

In [76]:
# Import Model for Weather Classification
import pickle
import os

model_dir = models_path


with open(os.path.join(model_dir, "xgb_weather_classification.pkl"), "rb") as f:
    model = pickle.load(f)

with open(os.path.join(model_dir, "feature_encoders.pkl"), "rb") as f:
    encoders = pickle.load(f)

with open(os.path.join(model_dir, "label_encoder_target.pkl"), "rb") as f:
    le_remark = pickle.load(f)



### Model Effective Capacity: Forecast effective_capacity_ton_day

- model: efcap_model
- encoder: efcap_encoders

In [77]:
# Load the encoders
efcap_encoders = joblib.load(f'{models_path}/encoders_efcap.pkl')

print(f"Label encoders loaded from: {models_path}/encoders_efcap.pkl")

Label encoders loaded from: /content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Modelling/Hasil Model/encoders_efcap.pkl


In [78]:
# Load Model
with open(f'{models_path}/effcap_ranfor_regression.pkl', 'rb') as f:
    efcap_model = pickle.load(f)

### Model Production Plan: Forecast Actual production ton
- model = prodplan_model
- encoder = prodplan_encoders


In [79]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [80]:
# Load the encoders
prodplan_encoders = joblib.load(f'{models_path}/encoders_prodplan.pkl')

print(f"Label encoders loaded from: {models_path}/encoders_prodplan.pkl")

Label encoders loaded from: /content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Modelling/Hasil Model/encoders_prodplan.pkl


In [81]:
# Load Model
prodplan_model = joblib.load(f'{models_path}/model_production_plan.pkl')

## Load Dataset (only for testing)


In [82]:
test_dir = f'{models_path}/testing_dataset'

In [83]:
weather_test = pd.read_csv(f'{test_dir}/weather_testing.csv')
efcap_test = pd.read_csv(f'{test_dir}/efcap_testing.csv')
prodplan_test = pd.read_csv(f'{test_dir}/prodplan_testing.csv')

In [84]:
efcap_test.head()

,mine_id,equipment_type,road_condition,weather_condition,availability_pct
0,MINE_3,Excavator,Good,Mendung,75
1,MINE_1,Dump Truck,Good,Mendung,99
2,MINE_1,Excavator,Good,Mendung,98
3,MINE_4,Excavator,Good,Mendung,87
4,MINE_2,Loader,Good,Mendung,71


In [85]:
prodplan_test.head()

,road_condition,weather_condition,availability_pct,effective_capacity_ton_day,planned_output_ton
0,Good,Mendung,75,256.09,77721
1,Good,Mendung,99,1519.87,64325
2,Good,Mendung,98,399.23,64325
3,Good,Mendung,87,218.14,59578
4,Good,Mendung,71,26.07,81761


## Predict Model

In [86]:
# Encoding mine_id
le = efcap_encoders['mine_id']

In [87]:
# Weather Forecast
weather_test['date'] = pd.to_datetime(weather_test['date'])
weather_pred = forecast_future(weather_test, models_forecast, feature_forecast, 7).head()
weather_pred.head()

,date,mine_id,mine_id_enc,month,week,dayofyear,rainfall_mm,temperature_c,humidity_pct,wind_speed_kmh
0,2025-11-08,MINE_1,0,11,45,312,0.493276,25.312319,68.717224,0.207723
1,2025-11-08,MINE_2,1,11,45,312,0.411516,25.129068,68.466515,0.263520
2,2025-11-08,MINE_3,2,11,45,312,0.668059,25.844196,64.655190,0.223234
3,2025-11-08,MINE_4,3,11,45,312,0.564971,25.558550,68.584282,0.292800
4,2025-11-08,MINE_5,4,11,45,312,0.552145,25.665813,68.584282,0.284201


In [88]:
weather_class_input = weather_pred.copy()
weather_class_input["date"] = weather_class_input["date"].astype("int64") / 1e9
weather_class_features_full = ["date", "rainfall_mm", "temperature_c", "wind_speed_kmh", "humidity_pct"]
weather_class = model.predict(weather_class_input[weather_class_features_full])
print("OK: weather_class")

OK: weather_class


In [89]:
# Effective Capacity
cat_cols_for_pred = efcap_test.select_dtypes(include=['object']).columns

for col in cat_cols_for_pred:
    if col in efcap_test.columns and col in efcap_encoders:
        # Use the loaded encoder to transform the column
        efcap_test[col] = efcap_encoders[col].transform(efcap_test[col])
    else:
        print(f"Warning: Column '{col}' not found in prediction DataFrame or encoder not loaded.")

print("Encoded DataFrame for prediction:")
efcap_test.head()

Encoded DataFrame for prediction:


,mine_id,equipment_type,road_condition,weather_condition,availability_pct
0,2,1,1,4,75
1,0,0,1,4,99
2,0,1,1,4,98
3,3,1,1,4,87
4,1,2,1,4,71


In [90]:
efcap_pred = efcap_model.predict(efcap_test)

In [91]:
prodplan_test

,road_condition,weather_condition,availability_pct,effective_capacity_ton_day,planned_output_ton
0,Good,Mendung,75,256.09,77721
1,Good,Mendung,99,1519.87,64325
2,Good,Mendung,98,399.23,64325
3,Good,Mendung,87,218.14,59578
4,Good,Mendung,71,26.07,81761
5,Fair,Mendung,86,50.64,77721
6,Good,Mendung,74,63.66,84764


In [92]:
# Production Plan
prodplan_test['road_condition'] = prodplan_encoders[0].transform(prodplan_test['road_condition'])
prodplan_test['weather_condition'] = prodplan_encoders[1].transform(prodplan_test['weather_condition'])
prodplan_pred = prodplan_model.predict(prodplan_test)

# Models+Agentic AI

### Input for Agentic AI

In [27]:
import pandas as pd
import joblib
import json
import openai

In [28]:
!pip install openai

In [ ]:
pip install --upgrade openai

In [93]:
weather_class_categorical = le_remark.inverse_transform(weather_class)
print("Categorical Weather Classification:")
print(weather_class_categorical)

Categorical Weather Classification:
['Mendung' 'Mendung' 'Mendung' 'Mendung' 'Mendung']


In [94]:
weather_output = pd.concat([weather_pred, pd.DataFrame(weather_class_categorical, columns=["weather_classification"])], axis=1)
weather_output.head()

,date,mine_id,mine_id_enc,month,week,dayofyear,rainfall_mm,temperature_c,humidity_pct,wind_speed_kmh,weather_classification
0,2025-11-08,MINE_1,0,11,45,312,0.493276,25.312319,68.717224,0.207723,Mendung
1,2025-11-08,MINE_2,1,11,45,312,0.411516,25.129068,68.466515,0.263520,Mendung
2,2025-11-08,MINE_3,2,11,45,312,0.668059,25.844196,64.655190,0.223234,Mendung
3,2025-11-08,MINE_4,3,11,45,312,0.564971,25.558550,68.584282,0.292800,Mendung
4,2025-11-08,MINE_5,4,11,45,312,0.552145,25.665813,68.584282,0.284201,Mendung


In [95]:
efcap_output = pd.concat([efcap_test, pd.DataFrame(efcap_pred, columns=["capacity_forecast"])], axis=1)
efcap_output.head()

,mine_id,equipment_type,road_condition,weather_condition,availability_pct,capacity_forecast
0,2,1,1,4,75,168.335742
1,0,0,1,4,99,1578.133550
2,0,1,1,4,98,164.668962
3,3,1,1,4,87,217.100373
4,1,2,1,4,71,50.837245


In [97]:
prodplan_output = pd.concat([prodplan_test, pd.DataFrame(prodplan_pred, columns=["production_plan"])], axis=1)
prodplan_output.head()

,road_condition,weather_condition,availability_pct,effective_capacity_ton_day,planned_output_ton,production_plan
0,1,4,75,256.09,77721,61148.52
1,1,4,99,1519.87,64325,49471.25
2,1,4,98,399.23,64325,49476.74
3,1,4,87,218.14,59578,45517.43
4,1,4,71,26.07,81761,58930.55


In [98]:
ml_output = {
    "weather_forecast": weather_output.to_dict(orient="records"),
    "capacity_forecast": efcap_output.to_dict(),
    "production_plan": prodplan_output.to_dict()
}

In [99]:
def convert_timestamps(obj):
    if isinstance(obj, pd.Timestamp):
        return obj.strftime("%Y-%m-%d")
    if isinstance(obj, dict):
        return {k: convert_timestamps(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [convert_timestamps(i) for i in obj]
    return obj

In [100]:
ml_output_clean = convert_timestamps(ml_output)

In [101]:
json_schema = """
{
   "recommendations": [
      {
         "title": "",
         "action": "",
         "justification": "",
         "expected_impact": ""
      }
   ]
}
"""

### OpenAI as Agentic AI

In [148]:
from dotenv import load_dotenv
import os

load_dotenv('/content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Modelling/ai_api_key_new.env', override=True)
api_key = os.getenv("OPENAI_API_KEY")

In [144]:
from openai import OpenAI
import json

client = OpenAI(api_key= api_key)

prompt = f"""
You are an expert mining operation optimization agent.

You will receive:
1. Weather forecast from an XGBoost model
2. Weather classification output
3. Effective capacity prediction
4. Actual production prediction
5. Known constraints (equipment, shipping, haulage, etc.)

Your task:
- Analyze all data holistically
- Generate a minimum of **5 actionable recommendations**
- Every recommendation MUST include:
  - **What to do**
  - **Why it matters (justification)**
  - **Expected impact (qualitative or quantitative)**
- Output format MUST be valid JSON with the structure:

JSON Schema:
{json_schema}


Be specific, realistic, and operationally useful.

Very important:
- DO NOT include explanations outside the JSON.
- DO NOT add comments, markdown, or introductory text.
- Only output valid JSON.
- Write all contents **inside the JSON** in **formal Indonesian**, optimized for mining engineers and operational planners.

Here is the data you must analyze:
{json.dumps(ml_output_clean, indent=2)}

"""

# 3. Panggil Agentic AI
response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": "You are a mining optimization agent."},
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)


{
  "recommendations": [
    {
      "title": "Penyesuaian Jadwal Pergeseran Kerja Berdasarkan Prediksi Cuaca Mendung",
      "action": "Optimalkan jadwal pergantian shift operator dan peralatan utama untuk memanfaatkan jam kerja efektif selama cuaca mendung dengan intensitas hujan rendah di semua tambang pada tanggal 2025-11-08.",
      "justification": "Cuaca mendung dengan curah hujan <1 mm dan kecepatan angin sangat rendah diprediksikan di seluruh area tambang, sehingga risiko gangguan kerja karena faktor cuaca sangat minim dan waktu kerja efektif dapat dimaksimalkan.",
      "expected_impact": "Produktivitas meningkat 5-10% pada seluruh front kerja dengan minimnya downtime akibat perubahan cuaca mendadak."
    },
    {
      "title": "Penjadwalan Ulang Pemeliharaan Alat Berat dengan Tingkat Ketersediaan Rendah",
      "action": "Segera lakukan preventive maintenance pada alat berat di lokasi dengan availability di bawah 80% (mine_id 0, 4, 6) ketika waktu henti tidak berdampak pada

In [145]:
output_ai_dir = '/content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Output JSON Rekomendasi AI'

In [146]:
import json
import os

ai_response_content = response.choices[0].message.content

# Ensure the output directory exists
os.makedirs(output_ai_dir, exist_ok=True)

# Define the output file path
output_file_path = os.path.join(output_ai_dir, 'recommendations.json')

# Parse the content and save it as a JSON file
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(json.loads(ai_response_content), f, ensure_ascii=False, indent=2)

print(f"AI recommendations saved to: {output_file_path}")

AI recommendations saved to: /content/drive/MyDrive/Asah-Capstone Project!/Dataset Thingy/Output JSON Rekomendasi AI/recommendations.json


### dependency

In [147]:
!pip freeze > requirements.txt